# HSE Summer School NLP&DA

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [26]:
def loadAnswer(number):
    with open("Texts/art{}.opin.txt".format(number)) as f:
        d = f.read()
    return d

def loadText(number):
    with open("Texts/art{}.txt".format(number)) as f:
        d = f.read()
    return d

In [160]:
def readAnnotation(number):

    with open("Texts/art{}.ann".format(number)) as f:
        d = f.read()

    d = d.split("\n")

    for i in range(len(d)):
        d[i] = d[i].split("\t")

    d = pd.DataFrame(d)
    d.drop([0], axis=1, inplace=True)
    d = pd.concat([d, pd.DataFrame(d[1].apply(lambda x: x.split()).tolist())], axis=1)
    d.columns = ["to_delete", "entity", "entity_car", "pos_1", "pos_2"]
    d.drop(["to_delete"], axis=1, inplace=True)
    d["entity"] = d["entity"].apply(lambda x: x.strip("\r"))
    return d

In [178]:
d[d.entity!="Unknown"].head(20)

,entity,entity_car,pos_1,pos_2
0,Author,PER,1,7
2,Author,PER,22,28
4,Россия,LOC,39,45
5,Сирии,LOC,72,77
6,Author,PER,81,87
8,Author,PER,147,153
10,Россия,LOC,164,170
11,Сирии,LOC,195,200
12,Author,PER,249,255
14,Author,PER,299,305


In [38]:
print(loadText(2))

{Author, Unknown} 

{Author, Unknown} Россия и кратковременный успех в Сирии

{Author, Unknown} inosmi.ruПосмотреть оригиналапрель 5-го, 2016

{Author, Unknown} Россия реализовала свои цели в Сирии, по крайней мере в краткосрочной перспективе. 
{Author, Unknown} Что будет дальше, пока неясно. 
{Author, Unknown} Точно можно сказать одно: подавляющее большинство сирийских граждан не смогут забыть российского участия в войне, которая была направлена против всего сирийского народа.


{Author, Unknown} Что касается Москвы, то будущее Сирии пока слабо представляется без участия России, особенно если принять во внимание, что сирийский конфликт имеет три уровня: иранский, американский и израильский.


{Author, Unknown} Иран окончательно смирился с тем, что судьба Башара Асада полностью зависит от решения Кремля. 
{Author, Unknown} Россия была призвана спасти то, что осталось от алавитского сирийского режима, сохранение которого представляет для Ирана особую важность.


{Author, Unknown} Если б

In [41]:
print(loadAnswer(2))

Сирия, Россия, pos, current
США, Асад, neg, past
Москва, Израиль, pos, current
Израиль, Россия, pos, current
Сирия, Израиль, neg, past
Израиль, Сирия, neg, past
Иран, Россия, pos, current
Россия, Иран, pos, current
Россия, Израиль, pos, current
Россия, Асад, pos, current
Author, Хафез Асад, neg, current
Author, Башар Асад, neg, current
Author, Израиль, pos, current
Иран, Башар Асад, pos, current

